In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 

In [2]:
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

cSCC_counts.txt file is a merged raw count matrix of all patients (after 10x cellranger processing and filtered: cells with <200 genes or >10% mitochondiral gene expression were rm).

Thus, separate by patient and output the saved file:

In [3]:
# cSCC_counts = pd.read_csv(path+'scRNAseq/'+'GSE144236_cSCC_counts.txt', sep = '\t', chunksize=1000)

# # merge all chunks into one df
# scRNAseq = pd.concat([chunk for chunk in cSCC_counts])

# # split df by patient
# cSCC_scRNA = []
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     cSCC_scRNA.append(j)
    
# # save df by patient idex to separate CSV files
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     j.to_csv('Data/squamous_carcinoma/scRNAseq_{}.csv'.format(i), header=True, index_label=False)

In [4]:
# squamous carcinoma dataset
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

all_counts = pd.read_csv(path+'ST/'+'GSE144239_ST_all_counts.txt', sep = '\t')
st_barcodes = pd.read_csv(path+'ST/'+'GSE144239_ST_barcode_id.txt', sep = '\t', header=None)
visium = pd.read_csv(path+'ST/'+'GSE144239_ST_Visium_counts.txt', sep = '\t')

metadata = pd.read_csv(path+'scRNAseq/'+'GSE144236_patient_metadata_new.txt', sep = '\t')

In [5]:
# read in all patient scRNAseq files
scRNA = [pd.read_csv(j, index_col=0) for j in glob.glob(os.path.join(path, '*.csv'))]

In [6]:
range(len(scRNA)) # scRNA for 10 patients

range(0, 10)

In [7]:
scRNA[0]

,P1_Tumor_AAACCTGAGTCAAGCG,P1_Tumor_AAACCTGCAAATTGCC,P1_Tumor_AAACCTGGTAGGAGTC,P1_Tumor_AAACGGGAGATGTAAC,P1_Tumor_AAACGGGAGCTGCAAG,P1_Tumor_AAAGATGAGAAGGTTT,P1_Tumor_AAAGATGCAATAACGA,P1_Tumor_AAAGATGCAGACTCGC,P1_Tumor_AAAGATGCAGTCAGCC,P1_Tumor_AAAGATGGTGGTGTAG,...,P1_Normal_TTGGCAAGTGTTCTTT,P1_Normal_TTGGCAATCCTGCTTG,P1_Normal_TTGTAGGAGGTGCTTT,P1_Normal_TTGTAGGCAAGAAGAG,P1_Normal_TTGTAGGTCCTGCTTG,P1_Normal_TTTACTGGTCTACCTC,P1_Normal_TTTATGCGTTTGTTGG,P1_Normal_TTTCCTCAGGAGCGAG,P1_Normal_TTTCCTCCACGGCGTT,P1_Normal_TTTCCTCTCACAGTAC
Patient,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
all_counts

,P2_7x45_1,P2_8x30_1,P2_8x32_1,P2_8x34_1,P2_8x36_1,P2_8x38_1,P2_8x44_1,P2_8x46_1,P2_9x29_1,P2_9x31_1,...,P5_41x27_3,P5_41x31_3,P5_41x33_3,P5_41x35_3,P5_41x37_3,P5_41x39_3,P5_41x41_3,P5_42x36_3,P5_42x38_3,P5_42x40_3
Patient,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Section replicate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
pixel_x,3282.6,3473.4,3471.9,3474.1,3469.5,3467.4,3471.4,3462.3,3650.9,3649.9,...,9448.4,9446.6,9437.3,9451.4,9451.1,9445.3,9451.1,9634.2,9628.1,9623.4
pixel_y,9536.7,6539.5,6937.9,7338.8,7736.4,8137.2,9337.8,9738.9,6336.8,6737.0,...,6984.0,7809.0,8205.3,8613.4,9011.1,9412.5,9810.7,8815.8,9224.7,9620.7
FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAGEA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MAGEA6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
LCE5A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NPIPA8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ST for P2, 5, 9 and 10 was generated with an older version of technology with larget spot size and center-to-center distance.

In [9]:
st_keep = np.unique(all_counts.loc['Patient'].values)
print(st_keep)
# type(st_keep)
st_keep = st_keep.tolist()
st_keep = [int(i) for i in st_keep] # turn float to int
print(st_keep)


[ 2.  5.  9. 10.]
[2, 5, 9, 10]


In [10]:
np.unique(all_counts.loc['Section replicate'].values)

array([1., 2., 3.])

In [11]:
print(type(scRNA))
print(type(scRNA[0]))

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


Keep only scRNA for patients 2, 5, 9, and 10

In [12]:
scrna = []
for i in range(len(scRNA)):
    if scRNA[i].loc['Patient'].isin(st_keep).any():
        scrna.append(scRNA[i])
len(scrna)

4

In [13]:
scrna[3]

,P9_Tumor_AAACCTGAGTGTACGG,P9_Tumor_AAACCTGCAGCTATTG,P9_Tumor_AAACCTGCATCCGGGT,P9_Tumor_AAACCTGTCGGAAACG,P9_Tumor_AAACGGGAGTGTCCCG,P9_Tumor_AAAGCAAAGATCGATA,P9_Tumor_AAAGTAGCACCATGTA,P9_Tumor_AAAGTAGTCTGGGCCA,P9_Tumor_AAATGCCCAGCTGGCT,P9_Tumor_AAATGCCGTCTTCGTC,...,P9_Normal_TTTGTCAAGCTGATAA,P9_Normal_TTTGTCAAGGTGCTTT,P9_Normal_TTTGTCAAGTTCGATC,P9_Normal_TTTGTCACAAAGCAAT,P9_Normal_TTTGTCACACAACGTT,P9_Normal_TTTGTCAGTTACTGAC,P9_Normal_TTTGTCATCAACACGT,P9_Normal_TTTGTCATCAGTCCCT,P9_Normal_TTTGTCATCGACAGCC,P9_Normal_TTTGTCATCTGTTTGT
Patient,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


10X Visium used for P4 and 6 (not at sc but better resolution than older version).

Note, spatial spot barcodes are also 16-oligo sequence.

In [14]:
visium

,P4_AAACAAGTATCTCCCA-1_1,P4_AAACATTTCCCGGATT-1_1,P4_AAACCTAAGCAGCCGG-1_1,P4_AAACGAGACGGTTGAT-1_1,P4_AAACGGGCGTACGGGT-1_1,P4_AAAGGGATGTAGCAAG-1_1,P4_AAAGTGTGATTTATCT-1_1,P4_AAAGTTGACTCCCGTA-1_1,P4_AAATAGGGTGCTATTG-1_1,P4_AAATCCGATACACGCC-1_1,...,P6_TTGTGTATGCCACCAA-1_2,P6_TTGTGTTTCCCGAAAG-1_2,P6_TTGTTAGCAAATTCGA-1_2,P6_TTGTTCAGTGTGCTAC-1_2,P6_TTGTTGGCAATGACTG-1_2,P6_TTGTTGTGTGTCAAGA-1_2,P6_TTGTTTCACATCCAGG-1_2,P6_TTGTTTCCATACAACT-1_2,P6_TTGTTTGTATTACACG-1_2,P6_TTGTTTGTGTAAATTC-1_2
Patient,4,4,4,4,4,4,4,4,4,4,...,6,6,6,6,6,6,6,6,6,6
Section replicate,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
AL627309.1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL669831.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC00115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AP001059.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP001469.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC136616.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC004556.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
st_barcodes

,0,1,2
0,GTCCGATATGATTGCCGC,2,6
1,GCGACGCGGCATAGTTTA,2,8
2,ACGTGCGCAAATATGACG,2,10
3,ATAAATGTTCACACTACT,2,12
4,TTGCTCGAATAAATAATT,2,14
...,...,...,...
1929,ATTAGTTATGCTGTTCAC,65,53
1930,GGTGAGACCTGAATTCCA,65,55
1931,CTATGTACTGTTCAAACC,65,57
1932,GGCTAGAACGAATAAACC,65,59


In [16]:
# split spot barcode on _ and keep just the ending
spt = []
for i in visium.columns:
    head, sep, tail = i.partition('_')
    spt.append(tail)

# split spot barcode on - and keep just first part
spot = []
for i in spt:
    spot.append(i.split('-', 1)[0])
np.unique(spot)

array(['AAACAAGTATCTCCCA', 'AAACACCAATAACTGC', 'AAACAGAGCGACTCCT', ...,
       'TTGTTTCCATACAACT', 'TTGTTTGTATTACACG', 'TTGTTTGTGTAAATTC'],
      dtype='<U16')

In [17]:
# count number of unique spots
len(np.unique(spot))

3993

In [18]:
spots = np.unique(spot).astype(object)
spots

array(['AAACAAGTATCTCCCA', 'AAACACCAATAACTGC', 'AAACAGAGCGACTCCT', ...,
       'TTGTTTCCATACAACT', 'TTGTTTGTATTACACG', 'TTGTTTGTGTAAATTC'],
      dtype=object)

In [19]:
st_barcodes.loc[:, 0]

0       GTCCGATATGATTGCCGC
1       GCGACGCGGCATAGTTTA
2       ACGTGCGCAAATATGACG
3       ATAAATGTTCACACTACT
4       TTGCTCGAATAAATAATT
               ...        
1929    ATTAGTTATGCTGTTCAC
1930    GGTGAGACCTGAATTCCA
1931    CTATGTACTGTTCAAACC
1932    GGCTAGAACGAATAAACC
1933    TGGGTGCTCATGTTCCGA
Name: 0, Length: 1934, dtype: object

In [20]:
st_barcodes.loc[:, 0].isin(spots)

0       False
1       False
2       False
3       False
4       False
        ...  
1929    False
1930    False
1931    False
1932    False
1933    False
Name: 0, Length: 1934, dtype: bool

In [21]:
st_barcodes[st_barcodes.loc[:, 0].isin(spots)]

,0,1,2


In [42]:
for i in st_barcodes[0]:
    for j in range(len([spt for spt in spots if i in spt])):
        if [spt for spt in spots if i in spt][j] > 0:
            print([spt for spt in spots if i in spt])
    # else:
    #         print("barcodes not in spots")

In [50]:
# another way to do the same thing as cell above
# for i in range(len(spot)):
#     if spot[i] in st_barcodes[0] == True:
#         print(spot[i])
#     # else:
#     #     print('no spot in barcode')

In [44]:
for i in spots:
    print([barc for barc in st_barcodes[0] if i in barc])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [40]:
for i in spots:
    for j in range(len([barc for barc in st_barcodes[0] if i in barc])):
        if [barc for barc in st_barcodes[0] if i in barc][j] > 0:
            print([[barc for barc in st_barcodes[0] if i in barc]])
        else:
            print("spot not in barcode")

AAACAAGTATCTCCCA
AAACACCAATAACTGC
AAACAGAGCGACTCCT
AAACAGCTTTCAGAAG
AAACAGGGTCTATATT
AAACAGTGTTCCTGGG
AAACATGGTGAGAGGA
AAACATTTCCCGGATT
AAACCCGAACGAAATC
AAACCGGGTAGGTACC
AAACCGTTCGTCCAGG
AAACCTAAGCAGCCGG
AAACCTCATGAAGTTG
AAACGAAGAACATACC
AAACGAAGATGGAGTA
AAACGAGACGGTTGAT
AAACGGGCGTACGGGT
AAACGGTTGCGAACTG
AAACTCGGTTCGCAAT
AAACTGCTGGCTCCAA
AAACTTAATTGCACGC
AAACTTGCAAACGTAT
AAAGAATGACCTTAGA
AAAGAATGTGGACTAA
AAAGACCCAAGTCGCG
AAAGACTGGGCGCTTT
AAAGGCCCTATAATAC
AAAGGCTACGGACCAT
AAAGGCTCTCGCGCCG
AAAGGGATGTAGCAAG
AAAGGGCAGCTTGAAT
AAAGTAGCATTGCTCA
AAAGTCACTGATGTAA
AAAGTCGACCCTCAGT
AAAGTGTGATTTATCT
AAAGTTGACTCCCGTA
AAATAACCATACGGGA
AAATAAGGTAGTGCCC
AAATACCTATAAGCAT
AAATAGCTTAGACTTT
AAATAGGGTGCTATTG
AAATCCGATACACGCC
AAATCGCGGAAGGAGT
AAATCGTGTACCACAA
AAATCTAGCCCTGCTA
AAATCTGCCCGCGTCC
AAATGACTGATCAAAC
AAATGATTCGATCAGC
AAATGCTCGTTACGTT
AAATGGCATGTCTTGT
AAATGGCCCGTGCCCT
AAATGGTCAATGTGCC
AAATGTATCTTATCCC
AAATGTGGGTGCTCCT
AAATTAACGGGTAGCT
AAATTAATAAGCGCGA
AAATTACACGACTCTG
AAATTACCTATCGATG
AAATTCCAGGTCCA

In [ ]:
print(np.unique(visium.iloc[0].values)) # find unique patients 
print(np.unique(visium.iloc[1].values)) # find unique replicates 

In [ ]:
metadata

In [ ]:
# count number of unique cells
barcds = [x.split('_')[2] for x in metadata.index] # split rownames on '_' and keep barcode only
np.unique(barcds).shape 

In [ ]:
metadata['patient'].unique()

In [ ]:
len(scRNA) # number of patients = 10 (as expected)